## Environment Setup


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense, Dropout
from PIL import Image
import os

In [ ]:
# Set up data generators with augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,  # Rotate by ±30°
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.2
)

# Initialize the ImageDataGenerator for testing (usually no augmentation for testing)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Setup paths
base_dir = '/content/drive/My Drive/Acnetify/DATASET_CLEANING/'

In [ ]:
batch_size = 16
epochs = 50
img_height, img_width = 224, 224

# Creating generators
train_generator = train_datagen.flow_from_directory(
    base_dir + 'Training_resize',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    # subset='training'  # Set as training data
)

validation_generator = train_datagen.flow_from_directory(
    base_dir + 'Val_resize',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    # subset='validation'  # Set as validation data
)

test_generator = test_datagen.flow_from_directory(
    base_dir + 'Test_crop',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 3000 images belonging to 5 classes.
Found 600 images belonging to 5 classes.
Found 80 images belonging to 4 classes.


In [ ]:
from PIL import Image
import os

def check_images(s_dir, ext_list):
    bad_images = []
    for fldr in os.listdir(s_dir):
        sub_folder = os.path.join(s_dir, fldr)
        if os.path.isdir(sub_folder):
            for file in os.listdir(sub_folder):
                file_path = os.path.join(sub_folder, file)
                if os.path.splitext(file_path)[1].lower() not in ext_list:
                    continue
                try:
                    img = Image.open(file_path)  # open the image file
                    img.verify()  # verify that it is, in fact, an image
                except (IOError, SyntaxError) as e:
                    print('Bad file:', file_path)  # print out the names of corrupt files
                    bad_images.append(file_path)
    return bad_images

bad_files = check_images('/content/drive/My Drive/Acnetify/DATASET_CLEANING/Training_awal', ['.jpg', '.jpeg', '.png'])
print('Found bad files:', bad_files)

Found bad files: []


## Model Building

In [ ]:
def ModelNet(input_shape=(224, 224, 3), num_classes=5):
    X_input = Input(input_shape)

    # Conv Block 1
    X = Conv2D(64, (3, 3), strides=(2, 2), padding='same')(X_input)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)

    # Conv Block 2
    X = Conv2D(48, (3, 3), strides=(2, 2), padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    # Conv Block 3
    X = Conv2D(32, (3, 3), padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)

    # Conv Block 4
    X = Conv2D(16, (3, 3), padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    # Top Layers
    X = Flatten()(X)
    X = Dense(512, activation='relu')(X)
    X = Dense(256, activation='relu')(X)
    X = Dense(128, activation='relu')(X)
    X = Dense(num_classes, activation='softmax')(X)

    model = Model(inputs=X_input, outputs=X, name='ModelNet')

    return model


In [ ]:
model = ModelNet(input_shape=(224, 224, 3), num_classes=5)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, steps_per_epoch=train_generator.samples // batch_size, epochs=epochs,
          validation_data=validation_generator, validation_steps=validation_generator.samples // batch_size)

Epoch 1/50
 31/187 [===>..........................] - ETA: 15:20 - loss: 1.6205 - accuracy: 0.2903

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))  # Use the length of accuracy array

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.tight_layout()  # Adjust the spacing between subplots
plt.show()

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

# Path untuk data pengujian
test_dir = '/content/drive/My Drive/Acnetify/DATASET_CLEANING/'

# Membuat objek ImageDataGenerator untuk data pengujian
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

# Evaluasi model menggunakan data pengujian
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

# Prediksi menggunakan data pengujian
predictions = model.predict(test_generator, steps=len(test_generator))
predicted_classes = np.argmax(predictions, axis=1)

# Menampilkan beberapa prediksi secara acak
random_indices = np.random.choice(len(predicted_classes), size=15, replace=False)
for i, idx in enumerate(random_indices):
    print(f'Image {i+1}: Predicted class - {predicted_classes[idx]}, True class - {test_generator.classes[idx]}')

# Menghitung confusion matrix
conf_matrix = confusion_matrix(test_generator.classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Menghitung dan mencetak classification report
class_names = list(test_generator.class_indices.keys())
report = classification_report(test_generator.classes, predicted_classes, target_names=class_names)
print("Classification Report:")
print(report)

# Menghitung dan mencetak accuracy score
accuracy = accuracy_score(test_generator.classes, predicted_classes)
print(f"Accuracy: {accuracy}")

# Menghitung dan mencetak F1 score
f1 = f1_score(test_generator.classes, predicted_classes, average='weighted')
print(f"F1 Score: {f1}")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7bff57b2-cd1b-4d31-87b2-4f5483361437' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>